### Search Engine With Tools And Agents

In [2]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [3]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [4]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [5]:
tools=[wiki,arxiv]

In [6]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings 
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, OllamaEmbeddings(model="jina/jina-embeddings-v2-small-en"))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x123b4b3b0>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [10]:
tools=[wiki,arxiv,retriever_tool]

In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/ramchandraregmi/Documents/AgenticAI/AgenticAIWorkspace/venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x117211da0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS o

In [12]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [13]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [15]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [16]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Skip to main conten

{'input': 'Tell me about Langsmith',
 'output': "You've already obtained the information about Langsmith. I'll summarize it for you:\n\nLangsmith is a platform for building production-grade LLM (Large Language Model) applications. It allows users to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. The platform provides features such as observability, evaluation, and prompt engineering.\n\nObservability enables users to analyze traces, configure metrics, and set up alerts. Evaluation allows users to assess the performance of their applications over production traffic and receive human feedback on their data. Prompt engineering helps users iterate on prompts with automatic version control and collaboration features.\n\nThe platform also provides a set of tools for building and running high-quality evaluations, including off-the-shelf evaluators and human feedback mechanisms. Langsmith integrates seamlessly with LangChain's open source fr

In [18]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': ' supervised learning'}`


Page: Supervised learning
Summary: In machine learning, supervised learning (SL) is a paradigm where a model is trained using input objects (e.g. a vector of predictor variables) and desired output values (also known as a supervisory signal), which a
Invoking: `langsmith-search` with `{'query': 'What is the difference between supervised and unsupervised learning'}`


LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-

{'input': 'What is machine learning',
 'output': 'The provided text does not seem to directly answer the question of what is the difference between supervised and unsupervised learning. It appears to be a marketing page for LangSmith, a platform for building production-grade LLM applications. Since the tool did not yield a relevant response, I will respond directly to your question.\n\nSupervised learning is a type of machine learning where the model is trained on labeled data, meaning the data has been hand-labeled or annotated with the correct output or class labels. The model learns to map inputs to outputs based on these labeled examples. The goal is to make predictions on new, unseen data.\n\nUnsupervised learning, on the other hand, is a type of machine learning where the model is trained on unlabeled data. The model discovers patterns, relationships, or structure in the data without knowing what the correct outputs or class labels are. The goal is to identify hidden features, gr

In [19]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" introduced the transformer architecture, which has since become a dominant sequence transduction model.

> Finished chain.


{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper "Attention Is All You Need" introduced the transformer architecture, which has since become a dominant sequence transduction model.'}